In [149]:
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [53]:
skel_path = '../quantitative_prox/recordings/vicon_03301_01/Skeleton/'
import glob
files = glob.glob(skel_path + '*')

In [54]:
files

['../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s001_frame_00001__00.00.00.023.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s002_frame_00001__00.00.00.013.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s003_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s004_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s005_frame_00001__00.00.00.012.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s006_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s007_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s008_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s009_frame_00001__00.00.00.009.json',
 '../quantitative_prox/recordings/vicon_03301_01/Skeleton\\s010_frame_00001__00.00.00.009.json']

In [57]:
with open(files[1]) as fn:
    stuff = json.load(fn)

In [58]:
stuff

{'FloorClipPlane': [-0.0214117952, 0.9990976, 0.036681477, 0.9368897],
 'Bodies': [{'First': True,
   'Lean': [-0.050362464, 0.115295641],
   'ClippedEdges': {'Bottom': False,
    'Left': False,
    'Top': False,
    'Right': False},
   'Joints': {'SpineBase': {'Position': [0.165960461,
      -0.156394392,
      3.11151671],
     'State': 'Tracked',
     'Rotation': [0.0178251043, 0.9272303, -0.022719251, -0.3733767]},
    'SpineMid': {'Position': [0.170960292, 0.153823689, 3.09428859],
     'State': 'Tracked',
     'Rotation': [0.01839411, 0.9174886, -0.0222610738, -0.396712661]},
    'Neck': {'Position': [0.174204171, 0.456588238, 3.06594872],
     'State': 'Tracked',
     'Rotation': [0.0314215273, 0.905296743, -0.0532051139, -0.4202614]},
    'Head': {'Position': [0.1367045, 0.5942464, 3.0950532],
     'State': 'Tracked',
     'Rotation': [0.0, 0.0, 0.0, 0.0]},
    'ShoulderLeft': {'Position': [0.0654687062, 0.3303389, 3.24483919],
     'State': 'Tracked',
     'Rotation': [0.71084

In [120]:
len(joints)

25

In [119]:
list(joints.keys())

['SpineBase',
 'SpineMid',
 'Neck',
 'Head',
 'ShoulderLeft',
 'ElbowLeft',
 'WristLeft',
 'HandLeft',
 'ShoulderRight',
 'ElbowRight',
 'WristRight',
 'HandRight',
 'HipLeft',
 'KneeLeft',
 'AnkleLeft',
 'FootLeft',
 'HipRight',
 'KneeRight',
 'AnkleRight',
 'FootRight',
 'SpineShoulder',
 'HandTipLeft',
 'ThumbLeft',
 'HandTipRight',
 'ThumbRight']

In [71]:
joints = stuff['Bodies'][0]['Joints']
pos_rots = [(x['Position'], x['Rotation']) for x in joints.values()]

In [72]:
pos_rots[0]

([0.165960461, -0.156394392, 3.11151671],
 [0.0178251043, 0.9272303, -0.022719251, -0.3733767])

In [76]:
import open3d as o3d

In [75]:
open3d.geometry.PointCloud

open3d.cpu.pybind.geometry.PointCloud

In [78]:
help(o3d.io.read_point_cloud)

Help on built-in function read_point_cloud in module open3d.cpu.pybind.io:

read_point_cloud(...) method of builtins.PyCapsule instance
    read_point_cloud(filename, format='auto', remove_nan_points=False, remove_infinite_points=False, print_progress=False)
    Function to read PointCloud from file
    
    Args:
        filename (str): Path to file.
        format (str, optional, default='auto'): The format of the input file. When not specified or set as ``auto``, the format is inferred from file extension name.
        remove_nan_points (bool, optional, default=False): If true, all points that include a NaN are removed from the PointCloud.
        remove_infinite_points (bool, optional, default=False): If true, all points that include an infinite value are removed from the PointCloud.
        print_progress (bool, optional, default=False): If set to true a progress bar is visualized in the console
    
    Returns:
        open3d.geometry.PointCloud



In [105]:
pcd = o3d.geometry.PointCloud()
xyz = np.array([[x[0] for x in pos_rots]]).reshape(-1, 3)
quats = np.array([[x[1] for x in pos_rots]]).reshape(-1, 4)
pcd.points = o3d.utility.Vector3dVector(xyz)

In [82]:
o3d.visualization.draw_geometries([pcd])

In [83]:
import numpy as np
import open3d as o3d
import torch


# test whether GPU works
try:
  aa = torch.cuda.FloatTensor(3,3).normal_()
except:
  # raise OSError('-- cuda is not supported')
  pass

# specify device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


print('-- pytorch version={}'.format(torch.__version__))
print('-- device={}'.format(device))

smplx_model_path='../models_smplx_v1_1/models/'

import smplx  # SMPL-X; pip install smplx
from human_body_prior.tools.model_loader import load_model


def get_smplx_model(gender, device):
    body_model_path = smplx_model_path
    body_model = smplx.create(body_model_path, 
                          model_type='smplx',        ## smpl, smpl+h, or smplx?
                          gender=gender, ext='npz',  ## file format 
                          num_pca_comps=12,          ## MANO hand pose pca component
                          create_global_orient=True, 
                          create_body_pose=True,
                          create_betas=True,
                          create_left_hand_pose=True,
                          create_right_hand_pose=True,
                          create_expression=True, 
                          create_jaw_pose=True,
                          create_leye_pose=True,
                          create_reye_pose=True,
                          create_transl=True,
                          batch_size=1               ## how many bodies in a batch?
                          )
    body_model.eval()
    return body_model

smplx_neutral = get_smplx_model(gender='neutral', device=device)


def draw_bodies(gender, betas, thetas):
    bm = smplx_neutral


    ## read verts and face from smplx model
    params = {'betas': betas,
              'body_pose': thetas
              }


    verts = bm(return_verts=True, **params).vertices.detach().cpu().numpy()[0]
    faces = bm.faces

    ## put verts and face into open3d, and compute surface normal
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    body = o3d.geometry.TriangleMesh()
    body.vertices = o3d.utility.Vector3dVector(verts)
    body.triangles = o3d.utility.Vector3iVector(faces)
    body.vertex_normals = o3d.utility.Vector3dVector([])
    body.triangle_normals = o3d.utility.Vector3dVector([])
    body.compute_vertex_normals()
    o3d.visualization.draw_geometries([body, coord])
    
    return None


-- pytorch version=1.10.0
-- device=cpu


In [84]:
gender = 'neutral'
betas = torch.FloatTensor(1,10).zero_()
# thetas = vposer.decode(torch.FloatTensor(1,32).normal_(), output_type='aa')
thetas = torch.FloatTensor(1, 21, 3).zero_()

draw_bodies(gender, betas, thetas)

In [88]:
dir(smplx_neutral(return_full_pose=True))

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'betas',
 'body_pose',
 'expression',
 'full_pose',
 'get',
 'global_orient',
 'items',
 'jaw_pose',
 'joints',
 'keys',
 'left_hand_pose',
 'right_hand_pose',
 'transl',
 'v_shaped',
 'values',
 'vertices']

In [97]:
pose = smplx_neutral(return_full_pose=True).body_pose.detach().numpy()

In [98]:
pose = pose.reshape(21, 3)

In [100]:
pose

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [106]:
quats

array([[ 0.0178251 ,  0.9272303 , -0.02271925, -0.3733767 ],
       [ 0.01839411,  0.9174886 , -0.02226107, -0.39671266],
       [ 0.03142153,  0.90529674, -0.05320511, -0.4202614 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.7108467 , -0.52166974, -0.3446686 ,  0.32211995],
       [ 0.00442779,  0.10714639,  0.99167603,  0.07126574],
       [ 0.9467569 , -0.2384138 , -0.20959999,  0.05364702],
       [ 0.9508916 , -0.24291183, -0.1889537 , -0.03309702],
       [ 0.76338077,  0.51248163, -0.34133682, -0.19519634],
       [ 0.9421635 ,  0.11096896,  0.18622175, -0.25560746],
       [-0.47911966, -0.23655872,  0.8256013 ,  0.18129192],
       [-0.45846066, -0.12835845,  0.84244424,  0.2522414 ],
       [ 0.7502524 , -0.65504926, -0.04711846,  0.07623353],
       [ 0.4485762 ,  0.06210395,  0.88908917,  0.06665626],
       [-0.30771   , -0.2575778 ,  0.90994984, -0.10468742],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.56783   ,  0.

In [121]:
from pytorch3d import transforms
aa = transforms.quaternion_to_axis_angle(torch.tensor(quats))

gender = 'neutral'
betas = torch.FloatTensor(1,10).zero_()
# thetas = vposer.decode(torch.FloatTensor(1,32).normal_(), output_type='aa')
# thetas = torch.FloatTensor(1, 21, 3).zero_()

draw_bodies(gender, betas, aa[:21].reshape(1, 21, 3).float())

In [116]:
aa.float().dtype

torch.float32

In [114]:
betas.dtype

torch.float32

In [115]:
aa.astype(torch.float32)

AttributeError: 'Tensor' object has no attribute 'astype'

In [138]:
pkl_path = '../quantitative_prox/fittings/mosh/vicon_03301_01/results/'
import glob
folders = glob.glob(pkl_path + '*')

In [143]:
import pickle
files = glob.glob(folders[6] + '/*')
with open(files[0], 'rb') as fn:
    out = pickle.load(fn, encoding='latin1')

In [156]:
folders[6]

'../quantitative_prox/fittings/mosh/vicon_03301_01/results\\s007_frame_00001__00.00.00.009'

In [150]:
out['beta'].shape

(10,)

In [151]:
gender = 'neutral'
betas = torch.FloatTensor(1,10).zero_()
# thetas = vposer.decode(torch.FloatTensor(1,32).normal_(), output_type='aa')
# thetas = torch.FloatTensor(1, 21, 3).zero_()

draw_bodies(gender, torch.tensor(out['beta']).reshape(1, 10).float(), torch.tensor(out['body_pose']).reshape(1, 21, 3).float())